##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Visualização de dados com o Projetor de embeddings no TensorBoard

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tensorboard/tensorboard_projector_plugin.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tensorboard/tensorboard_projector_plugin.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tensorboard/tensorboard_projector_plugin.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Usando o **Projetor de embeddings do TensorBoard**, você pode representar embeddings de muitas dimensões graficamente. Isso pode ser útil ao visualizar, examinar e entender suas camadas de embeddings.

Neste tutorial, você aprenderá a visualizar esse tipo de camada treinada.

## Configuração

Para este tutorial, vamos usar o TensorBoard para visualizar uma camada de embeddings gerada para classificar dados de avaliações de filmes.

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

%load_ext tensorboard

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector


## Dados do IMDB

Vamos usar um dataset de 25 mil avaliações de filmes do IMDB, cada um com um rótulo de sentimento (positivo/negativo). Cada avaliação é pré-processada e codificada como uma sequência de índices de palavras (números inteiros). Para simplificar, as palavras são indexadas pela frequência geral no dataset, por exemplo, o inteiro "3" codifica a 3ª palavra que aparece com mais frequência em todas as avaliações. Isso permite filtrar rapidamente operações como: "só considere as 10 mil palavras mais comuns, mas elimine as 20 palavras mais comuns".

Como convenção, "0" não representa nenhuma palavra específica, mas, em vez disso, é usado para codificar qualquer palavra desconhecida. Mais tarde no tutorial, vamos remover a linha de "0" na visualização.


In [ ]:
(train_data, test_data), info = tfds.load(
    "imdb_reviews/subwords8k",
    split=(tfds.Split.TRAIN, tfds.Split.TEST),
    with_info=True,
    as_supervised=True,
)
encoder = info.features["text"].encoder

# Shuffle and pad the data.
train_batches = train_data.shuffle(1000).padded_batch(
    10, padded_shapes=((None,), ())
)
test_batches = test_data.shuffle(1000).padded_batch(
    10, padded_shapes=((None,), ())
)
train_batch, train_labels = next(iter(train_batches))


# Camada de embeddings do Keras

Uma [camada de embeddings do Keras](https://keras.io/layers/embeddings/) pode ser usada para treinar um embedding para cada palavra no seu vocabulário. Cada palavra (ou subpalavra, nesse caso) será associada a um vetor (ou embedding) de 16 dimensões que será treinado pelo modelo.

Veja [este tutorial](https://www.tensorflow.org/tutorials/text/word_embeddings?hl=en) para saber mais sobre embeddings de palavras.

In [9]:
# Create an embedding layer.
embedding_dim = 16
embedding = tf.keras.layers.Embedding(encoder.vocab_size, embedding_dim)
# Configure the embedding layer as part of a keras model.
model = tf.keras.Sequential(
    [
        embedding, # The embedding layer should be the first layer in a model.
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1),
    ]
)

# Compile model.
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Train model for one epoch.
history = model.fit(
    train_batches, epochs=1, validation_data=test_batches, validation_steps=20
)

2500/2500 [==============================] - 13s 5ms/step - loss: 0.5330 - accuracy: 0.6769 - val_loss: 0.4043 - val_accuracy: 0.7800


## Salve dados para o TensorBoard

O TensorBoard lê tensores e metadados de logs dos seus projetos do TensorFlow. O caminho para o diretório de log é especificado com `log_dir` abaixo. Para este tutorial, vamos usar `/logs/imdb-example/`.

Para carregar os dados no Tensorboard, precisamos salvar um checkpoint de treinamento para esse diretório, além dos metadados que permitem a visualização de uma camada específica de interesse no modelo. 

In [ ]:
# Set up a logs directory, so Tensorboard knows where to look for files.
log_dir='/logs/imdb-example/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for subwords in encoder.subwords:
    f.write("{}\n".format(subwords))
  # Fill in the rest of the labels with "unknown".
  for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
    f.write("unknown #{}\n".format(unknown))


# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(model.layers[0].get_weights()[0][1:])
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [ ]:
# Now run tensorboard against on log data we just saved.
%tensorboard --logdir /logs/imdb-example/

<!-- <img class="tfo-display-only-on-site" src="images/embedding_projector.png?raw=1"/> -->

## Análise

O Projetor do TensorBoard é uma ótima ferramenta para interpretar e visualizar embeddings. O painel de controle permite que os usuários pesquisem termos específicos e destaquem palavras adjacentes no espaço do embedding (poucas dimensões). Com esse exemplo, podemos ver que Wes **Anderson** e Alfred **Hitchcock** são termos bastante neutros, mas mencionados em contextos diferentes.

<!-- <img class="tfo-display-only-on-site" src="images/embedding_projector_hitchcock.png?raw=1"/> -->

Nesse espaço, Hitchcock está mais perto de palavras como `nightmare` (pesadelo), provavelmente devido ao fato de que ele é conhecido como o "Mestre do suspense", enquanto Anderson está mais próximo da palavra `heart` (coração), que é consistente com o estilo implacavelmente detalhado e comovente.

<!-- <img class="tfo-display-only-on-site" src="images/embedding_projector_anderson.png?raw=1"/> -->